In [2]:
import os
from glob import glob

In [10]:
import os

# 경로 설정
data_path = '/home/nute11a/workspace/2026_AI_Advanced_Study/3차시/03_Hand_Detection/data/labels/val'
ok_list = []
error_count = 0

# 전체 파일 순회
file_list = os.listdir(data_path)

for lname in file_list:
    if not lname.endswith('.txt'):
        continue
        
    file_path = os.path.join(data_path, lname)
    is_valid = True  # 현재 파일이 정상이라고 가정
    
    with open(file_path, 'r') as f:
        lines = f.readlines()
        
    for line in lines:
        parts = list(map(float, line.strip().split()))
        
        # parts 구조: [class, box_x, box_y, box_w, box_h, kpt_x1, kpt_y1, kpt_v1, ...]
        
        # 1. Bounding Box 검사 (index 1~4)
        # 박스 좌표는 무조건 0~1 사이여야 함
        if any(x < 0.0 or x > 1.0 for x in parts[1:5]):
            is_valid = False
            break
            
        # 2. Keypoints 검사 (index 5부터 끝까지)
        # 3개씩 묶음 (x, y, visibility) -> x, y만 검사해야 함! (visibility는 2가 나올 수 있음)
        keypoints = parts[5:]
        for i in range(0, len(keypoints), 3):
            # x 좌표
            kp_x = keypoints[i]
            # y 좌표 (배열 길이 초과 방지 체크)
            kp_y = keypoints[i+1] if i+1 < len(keypoints) else 0.5 
            
            # 좌표가 범위를 벗어나면 불량 처리
            if not (0.0 <= kp_x <= 1.0) or not (0.0 <= kp_y <= 1.0):
                is_valid = False
                break
        
        if not is_valid:
            break # 라인 하나라도 이상하면 파일 전체 스킵

    if is_valid:
        ok_list.append(lname.replace('.txt', ''))
    else:
        error_count += 1
        # print(f"Error file found: {lname}") # 필요시 주석 해제하여 확인

print(f"전체 파일: {len(file_list)}개")
print(f"정상 파일(ok_list): {len(ok_list)}개")
print(f"범위 벗어난 파일: {error_count}개")

전체 파일: 7992개
정상 파일(ok_list): 7802개
범위 벗어난 파일: 190개


In [ ]:
import shutil
# 2000개 랜덤 얻기
ok_list = ok_list[:2000]
# train:val:test = 6:2:2 비율로 복사
train_count = int(len(ok_list) * 0.6)
val_count = int(len(ok_list) * 0.2)
train_list = ok_list[:train_count]
val_list = ok_list[train_count:train_count + val_count]
test_list = ok_list[train_count + val_count:]

print(f"Train 파일: {len(train_list)}개")
print(f"Val 파일: {len(val_list)}개")
print(f"Test 파일: {len(test_list)}개")

Train 파일: 1200개
Val 파일: 400개
Test 파일: 400개


In [ ]:
for id in train_list:
    # 이미지 파일 복사
    src_img = f'images/val/{id}.jpg'
    dst_img = f'2000/images/train/{id}.jpg'
    shutil.copyfile(src_img, dst_img)
    # 라벨 파일 복사
    src_lbl = f'labels/val/{id}.txt'
    dst_lbl = f'2000/labels/train/{id}.txt'
    shutil.copyfile(src_lbl, dst_lbl)
    break

FileNotFoundError: [Errno 2] No such file or directory: 'labels/train/IMG_00012409.txt'